In [1]:
import chess
import math
import random
import torch
import torch.nn as nn
import torch.optim as optim
import dset
import net
import autoencoder
import bitboards

##Hyperparameter
c_const = 0.3
samplingRate = 0.4
seed = random.randint(0, 100)


mse = nn.MSELoss()

def cross_entropy(y_hat, y):    
    loss = nn.CrossEntropyLoss()   
    y_hat_concat = torch.cat((y_hat[0], y_hat[1]), 1)
    y = torch.reshape(y, (y.size()[0], 128))
            
    return loss(y_hat_concat, y)

def generate_dataset(dataset_size, transform, reinf_type, game_generator, *args):
    return dset.SearchDataset(dataset_size, transform, reinf_type, game_generator, *args)

def subset_of_dataset(dataset, sampling_rate):
    pick = math.floor(samplingRate*len(dataset))
    subset = torch.utils.data.random_split(dataset, [pick, len(dataset) - pick], generator=torch.Generator().manual_seed(seed))
    
    return subset

def get_dataloader(subset, batch):
    return torch.utils.data.DataLoader(subset[0], batch_size=batch, shuffle=True, drop_last=True)

def generate_trainable_data(batch, dataset_size, nnet, optimizer, reinf, game_generator, *args):
    dataset = generate_dataset(dataset_size, reinf, game_generator, *args)
    subset = subset_of_dataset(dataset, samplingRate)    
    DataLoader = get_dataloader(subset, batch)
    
    return DataLoader
    

def train_mcts(nnet, encoder, optimizer, data_loader, file="nnet_mcts.pt", file2="autoencoderftest2.pt"):    
    noBatch = 0
    running_loss, running_mse, running_cross_entropy = 0, 0, 0
    
    for position, value, policy in data_loader:        
        optimizer.zero_grad()
        
        embedding = encoder.encode(position.squeeze())
        value_hat, policy_hat = nnet(embedding.squeeze())
        mse_value = mse(value_hat, torch.tensor(value, dtype=torch.float).cuda())
        cross_entropy_value = cross_entropy(policy_hat, policy)
        loss = torch.tensor(c_const).cuda() * mse_value + torch.tensor(1.0 - c_const).cuda() * cross_entropy_value        
    
        running_loss += loss.item()
        running_mse += mse_value.item()
        running_cross_entropy += cross_entropy_value.item()
        
        loss.backward()
        optimizer.step()
        noBatch += 1
            
    print(f"Loss: \t", running_loss/noBatch, "\n\t\t Value loss: ", running_mse/noBatch, "\n\t\t Policy loss: ", running_cross_entropy/noBatch, end='\n\n')

    torch.save(nnet.state_dict(), file)
    torch.save(encoder.state_dict(), file2)
                
def train_alpha_beta(batch, dataset_size, encoder, nnet, optimizer, reinf, game_generator, *args):
    dataset = dset.SearchDataset(dataset_size, reinf, game_generator, *args)
    pick = math.floor(samplingRate*len(dataset))
    subset = torch.utils.data.random_split(dataset, [pick, len(dataset) - pick], generator=torch.Generator().manual_seed(seed))
    
    DataLoader = torch.utils.data.DataLoader(subset[0], batch_size=batch, shuffle=True, drop_last=True)
    
    noBatch = 0
    for embedding, value in DataLoader:
        optimizer.zero_grad()
        value_hat = nnet(embedding.view(embedding.shape[0],1, 256))

        mse_value = mse(value_hat, value.cuda())
        print(f"Loss ({noBatch}): ", mse_value.item(), end='\n')

        mse_loss.backward() 
        optimizer.step()
        noBatch += 1
        
    torch.save(nnet.state_dict(), "nnet_alpha_beta.pt")

In [2]:
encoder = autoencoder.autoencoder().cuda()
nnet = net.Net().cuda()

encoder.train()
nnet.train()

for param in encoder.parameters():
    param.requires_grad = True

params = list(encoder.parameters()) + list(nnet.parameters())
optimizer = optim.Adam(params, weight_decay=0.01)

##Hyperparameters 
BATCH = 16
DATASET_SIZE = 128
reinf = dset.ReinforcementType.PARAM
ARGS = (chess.Board(), nnet, encoder, dset.SearchType.CUSTOM, 5)
GameGenerator = dset.GameGenerator(1, 0, 1, dset.ReinforcementType.MC)
transform = None

for j in range(0, 100):
    dataset = generate_dataset(DATASET_SIZE, transform, reinf, GameGenerator, *ARGS)

    subset = subset_of_dataset(dataset, samplingRate)
    DataLoader = get_dataloader(subset, BATCH)

    train_mcts(nnet, encoder, optimizer, DataLoader)
    nnet.load_state_dict(torch.load("nnet_mcts.pt"))
    encoder.load_state_dict(torch.load("autoencoderftest2.pt"))

/home/mputo/.local/lib/python3.11/site-packages/torch/nn/modules/conv.py:137: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:77.)
  self.weight = Parameter(torch.empty(
/tmp/ipykernel_7343/3509321809.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mse_value = mse(value_hat, torch.tensor(value, dtype=torch.float).cuda())
/home/mputo/.local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss: 	 6.865535577138265 
		 Value loss:  0.24727338552474976 
		 Policy loss:  9.701933860778809

Loss: 	 6.855778058369954 
		 Value loss:  0.21492122610410055 
		 Policy loss:  9.701859792073568

Loss: 	 6.853382746378581 
		 Value loss:  0.2071957935889562 
		 Policy loss:  9.701748847961426

Loss: 	 6.850845177968343 
		 Value loss:  0.19910610218842825 
		 Policy loss:  9.701590538024902

Loss: 	 6.844990571339925 
		 Value loss:  0.1800259401400884 
		 Policy loss:  9.701404253641764



KeyboardInterrupt: 